In [1]:
system("conda install -y r-sf=0.9_8")

In [2]:
system("conda install -y r-terra=1.2_10")

In [3]:
system("conda install -y r-dplyr")

In [4]:
system("conda install -y r-stringr")

In [5]:
system("conda install -y r-RColorBrewer")

In [6]:
# 1 LOAD LIBRARIES ----
library(terra)
library(dplyr)
library(stringr)
library(RColorBrewer)
library(sf)
library(jsonlite)

# 2 VARIABLES ----
months_days <- list(
  Jan = 31, Feb = 28, Mar = 31, Apr = 30, May = 31, Jun = 30, 
  Jul = 31, Aug = 31, Sep = 30, Oct = 31, Nov = 30, Dec = 31
)


json_path <- "galaxy_inputs/galaxy_inputs.json"
json_data <- fromJSON(json_path)
subarea <- json_data$subarea
year <- json_data$year
year <- as.numeric(year)
month <- json_data$month

month <- match(month, names(months_days))

output_folder <- sprintf("%s_tif_daily", year)
dir.create(output_folder, showWarnings = FALSE, recursive = TRUE)

# 3 FUNCTION ---




library(lubridate)

download_tiff_files <- function(path, year, month) {
  
  base_url <- "https://noaadata.apps.nsidc.org/NOAA/G02135/south/daily/geotiff/"
  
  months <- c("01_Jan", "02_Feb", "03_Mar", "04_Apr", "05_May", "06_Jun",
              "07_Jul", "08_Aug", "09_Sep", "10_Oct", "11_Nov", "12_Dec")
  
  # Définir la date de départ
  start_date <- as.Date(sprintf("%04d-%02d-01", year, month))
  
  # Générer les 12 mois suivants
  dates <- seq.Date(start_date, by = "month", length.out = 12)
  dates_str <- as.character(dates)
  print(dates_str)
  
  for (date in dates_str) {
    year_str <- year(date) 
    month_num <-month(date)
    month_index <- sprintf("%02d", month_num)  # Format 01, 02, ..., 12
    month_folder <- months[month_num]  # Sélectionner le dossier du mois
    
    # Obtenir le nombre de jours dans le mois
    num_days <- days_in_month(date)
    
    for (day in 1:num_days) {
      date_str <- sprintf("S_%04d%02d%02d_concentration_v3.0.tif", year_str, month_num, day)
      url <- paste0(base_url, year_str, "/", month_folder, "/", date_str)
      destfile <- file.path(path, date_str)
      
      tryCatch({
        download.file(url, destfile, mode = "wb")
      }, error = function(e) {})
    }
  }
}


# 4 MAIN ----
# Télécharger les fichiers .tif pour l'année et le mois spécifiés
download_tiff_files(output_folder, year, month)

# Lister tous les fichiers .tif téléchargés
tif_files <- list.files(output_folder, pattern = paste0(year, "|", formatC(month, width = 2, flag = "0")), full.names = TRUE)

# Diviser les fichiers en deux sous-ensembles (selon les périodes que vous souhaitez traiter)
tif_files_subset_1 <- tif_files[1:length(tif_files)]

# Fonction pour traiter les fichiers .tif
x <- function(tif_files){
  stacked_rasters <- rast(tif_files)
  stacked_rasters[is.na(stacked_rasters)] <- 0
  reclassified_raster <- ifel(stacked_rasters >= 150 & stacked_rasters <= 1000, 1, 0)
  sum_raster <- sum(reclassified_raster, na.rm = TRUE)
  return(sum_raster)
}

# 5. LOAD AND MASK DATA (ASD) ----
output_dir_asd <- "asd"
dir.create(output_dir_asd, showWarnings = FALSE)

# URLs pour les shapefiles ASD
urls_asd <- c(
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.shp",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.shx",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG4326.prj",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.shp",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.shx",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.dbf",
  "https://raw.githubusercontent.com/ccamlr/data/refs/tags/v0.5.0/geographical_data/asd/asd-shapefile-EPSG6932.prj"
)

destfiles_asd <- file.path(output_dir_asd, basename(urls_asd))

for (i in seq_along(urls_asd)) {
  download.file(urls_asd[i], destfiles_asd[i], mode = "wb")
}

ASDs <- st_read(file.path(output_dir_asd, "asd-shapefile-EPSG4326.shp"))

# Reproject ASD to the correct CRS (EASE-Grid 2.0 South)
ASDs <- st_transform(ASDs, crs = 6932)

# 6. CREATE MASK ----

area_list <- unlist(strsplit(subarea, "[,]"))
area_list <- trimws(area_list)  # Supprime les espaces avant et après chaque élément

subareas_selected <- ASDs %>% filter(GAR_Short_ %in% area_list)

# Créer un bounding box pour la zone sélectionnée
bbox <- st_bbox(subareas_selected)
bbox_polygon <- st_as_sfc(bbox)
bbox_vect <- vect(bbox_polygon)

# Appliquer le masque sur les rasters
total_sum_raster_subset_1 <- x(tif_files_subset_1)

# CROP : Ne garder que la zone d'intérêt (au lieu de tout le raster)
total_sum_raster_subset_1_cropped <- crop(total_sum_raster_subset_1, bbox_vect)

# Combiner les deux rasters
total_sum_raster <- total_sum_raster_subset_1_cropped 

# Appliquer un masque (optionnel, mais peut être utile pour éviter des valeurs en dehors)
total_sum_raster_masked <- mask(total_sum_raster, bbox_vect)

# 7. PLOT ----
# Palette de couleurs
colors <- colorRampPalette(c("#2F133B", "#367AFC", "#00C3EA", "#03F791", "#72FE00","#FDC207", "#FF7504", "#F32106", "#7B0503"))(length(tif_files))

# Créer la carte
png("outputs/ice_days_map_masked.png", width = dim(total_sum_raster_masked)[2]*20, height = dim(total_sum_raster_masked)[1]*20)

# Tracer le raster masqué sans légende
plot(total_sum_raster_masked, col = colors, axes = FALSE, box = FALSE, legend = FALSE, mar=c(0,0,0,0))
plot(st_geometry(subareas_selected), add = TRUE, lwd = 2, border = 'white',mar=c(0,0,0,0))

zones <- st_read("asd/asd-shapefile-EPSG6932.shp")    
centroids <- st_centroid(st_geometry(zones))
text(st_coordinates(centroids), labels = zones$GAR_Long_L, col = "white", cex = 4) 

dev.off()



Warning message:
“package ‘terra’ was built under R version 4.0.5”
terra version 1.2.10


Attaching package: ‘dplyr’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘RColorBrewer’ was built under R version 4.0.5”
Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0


Attaching package: ‘lubridate’


The following objects are masked from ‘package:terra’:

    intersect, union


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




 [1] "2023-04-01" "2023-05-01" "2023-06-01" "2023-07-01" "2023-08-01"
 [6] "2023-09-01" "2023-10-01" "2023-11-01" "2023-12-01" "2024-01-01"
[11] "2024-02-01" "2024-03-01"
Reading layer `asd-shapefile-EPSG4326' from data source `/data/jwd02f/pulsar_staging/79838529/working/jupyter/asd/asd-shapefile-EPSG4326.shp' using driver `ESRI Shapefile'
Simple feature collection with 19 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -180 ymin: -85.41189 xmax: 180 ymax: -45
Geodetic CRS:  WGS 84


Warning message in x@ptr$mask_vector(mask@ptr, inverse[1], updatevalue[1], touches[1], :
“GDAL Message 1: Failed to fetch spatial reference on layer lyr to build transformer, assuming matching coordinate systems.”
Warning message in plot.window(...):
“"box" is not a graphical parameter”
Warning message in plot.xy(xy, type, ...):
“"box" is not a graphical parameter”
Warning message in title(...):
“"box" is not a graphical parameter”


Reading layer `asd-shapefile-EPSG6932' from data source `/data/jwd02f/pulsar_staging/79838529/working/jupyter/asd/asd-shapefile-EPSG6932.shp' using driver `ESRI Shapefile'


Warning message in CPL_read_ogr(dsn, layer, query, as.character(options), quiet, :
“GDAL Error 1: unhandled axis direction: "North along 90 deg East"”


Simple feature collection with 19 features and 16 fields
Geometry type: POLYGON
Dimension:     XY
Bounding box:  xmin: -3348556 ymin: -3329339 xmax: 4815055 ymax: 4371127
CRS:           NA


png 
  2